# APIM ‚ù§Ô∏è OpenAI

## Developer tooling lab
![flow](../../images/developer-tooling.gif)

Playground to try the developer tooling available with APIM to develop, debugg, test and publish AI Service APIs.

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

### 0Ô∏è‚É£ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [ ]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"

openai_resources = [
    {"name": "openai1", "location": "swedencentral"},
    {"name": "openai2", "location": "francecentral"}
]
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"


### 1Ô∏è‚É£ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [1]:
# %load ../../shared/snippets/create-az-resource-group.py

# type: ignore

import datetime

resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print(f"‚úÖ Azure Resource Group {resource_group_name} created ‚åö {datetime.datetime.now().time()}")


### 2Ô∏è‚É£ Create deployment using ü¶æ Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [2]:
# %load ../../shared/snippets/create-az-deployment.py

# type: ignore

import json

backend_id = "openai-backend-pool" if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_xml = policy_xml_file.read()

    if backend_id:
        policy_xml = policy_xml.replace("{backend-id}", backend_id)

    if client_id:
        policy_xml = policy_xml.replace("{aad-client-application-id}", client_id)

    if tenant_id:
        policy_xml = policy_xml.replace("{aad-tenant-id}", tenant_id)

    policy_xml_file.close()
open("policy-updated.xml", 'w').write(policy_xml)

bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"


### 3Ô∏è‚É£ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [18]:
# %load -r 1-14 ../../shared/snippets/deployment-outputs.py

# type: ignore

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimServiceId.value -o tsv
apim_service_id = stdout.n
print(f"üëâüèª APIM Service Id: {apim_service_id}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = stdout.n
print(f"üëâüèª APIM Subscription Key (masked): ****{apim_subscription_key[-4:]}")

stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = stdout.n
print(f"üëâüèª APIM API Gateway URL: {apim_resource_gateway_url}")

### üß™ Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [ ]:
import requests

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

messages={ "messages": [
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]}

response = requests.post(url, headers = {'api-key': apim_subscription_key}, json = messages)
print(f"status code: {response.status_code}")
print(f"headers: {response.headers}")

if "x-ms-region" in response.headers:
    print(f"x-ms-region: \x1b[1;31m{response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request

if (response.status_code == 200):
    data = json.loads(response.text)
    print(f"response: {data.get("choices")[0].get("message").get("content")}")
else:
    print(response.text)


### üß™ Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [ ]:
from openai import AzureOpenAI

messages = [
    {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
    {"role": "user", "content": "Can you tell me the time, please?"}
]

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription_key,
    api_version = openai_api_version
)

response = client.chat.completions.create(model=openai_model_name, messages=messages)
print(response.choices[0].message.content)

### üóëÔ∏è Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.